# Electiva Técnica I - Introducción Robot Operating System (ROS1)¶

## Construcción de Paquetes y Nodos

### David Rozo Osorio, I.M, M.Sc.

## Introducción a la implementación de Nodos

- Objetivo: diseñar e implementar un *node* que envíe mensajes al simulador Turtlesim


- Procedimiento:
 1. Creación de paquetes.
 2. Creación de un nodo mínimo.
 3. Nodo Subscriptor-Publicador Teleop
 4. Ejecución.
 5. Nodo Subscriptor-Publicador Joy
 6. Ejecución.

### Creación del Espacio de Trabajo (*workspace*) un paquete (*pkg*)

1. Ubicado en `Home/User` crear la carpeta asociada para el *Workspace*
```bash
~$ mkdir -p <worksspace_name>/src 
```
 **Ej.** La carpeta se llama `catkin_ws` y esta ubicada en `Home/User`:
```bash
~$ mkdir -p ~/catkin_ws/src 
```

2. Inicializando el *Workspace* desde la carpeta `/src`:
```bash
~/catkin_ws/src $ catkin_init_workspace 
```

3. Ubicado en la raíz del espacio de trabajo `catkin_ws`:
```bash
~/catkin_ws $ catkin_make 
```

4. Agregar el espacio de trabajo al archivo `.bashrc`:
```bash
$ echo "source ~/catkin_ws/devel/setup.bash" >> ~/.bashrc
$ source ~/.bashrc
```

5. Creación del paquete, dentro de la carperta `src` (`catkin_ws/src`):
```bash
~/catkin_ws/src $ catkin_create_pkg <pkg_name> <pkg_deps>
```
 **Nota**: el `<pkg_name>` por convención todo en minúsculas y si es necesario separado por guion bajo.
 
 **Ej.** el paquete se llamará `hello_world`, con dependencias de: C++, Python y mensajes estándar de ROS:
```bash
~/catkin_ws/src $ catkin_create_pkg hello_world roscpp rospy std_msgs
```

 **Ej.** el paquete se llamará `turtle_pkg_gYY`, con dependencias de: C++, Python y mensajes estándar de ROS:
```bash
~/catkin_ws/src $ catkin_create_pkg turtle_pkg_gYY roscpp rospy std_msgs
```

 **Nota**: el `YY` debe coincidir con la desiganación de Grupo.


6. Iniciar sesión de Visual Studio Code desde la carpeta `catkin_ws`:

```bash
~/catkin_ws $ code .
```

### Paquetes requeridos para la sesión

- Teleop Twist Keyboard

```bash
~$ sudo apt-get install ros-noetic-teleop-twist-keyboard
```

- Joystick

```bash
~$ sudo apt-get install ros-noetic-joy
```

## Teleoperación con teclado

1. Terminal No. 1 Inicializar el ROS-Master

```bash
$ roscore
```

2. Terminal No. 2. Iniciar nodo para lectura de teclado.

```bash
$ rosrun teleop_twist_keyboard teleop_twist_keyboard
```

3. Terminal No. 3. Leer *node_info* y *topic_info*


### Nodo turtle_teleop (Python)

- Copiar el **nodo** `turtle_node_g00.py` asignarle el nombre `turtle_node_g00_teleop.py`

- Inclusión de librerías en el programa.

```python
from geometry_msgs.msg import Twist

# def twist variable
keyboard_cmd = Twist()
```

- En la parte incial de la función del *node*, crear el subscriptor al nodo teleop.

```python
cmdkey_pub = rospy.Subscriber('/cmd_vel',Twist,twist_callback)
```

- Creación de una función (*method*) encargada de leer el mensaje (`twist_callback`).

```python
def twist_callback(data):
    global keyboard_cmd
    keyboard_cmd = data
    rospy.loginfo(keyboard_cmd)
```

- Modificar la estructura del `While` para que contruya y envíe el mensaje.

```python
def turtle_node_g00(): #YY must change
#...
global keyboard_cmd
while not rospy.is_shutdown():
    #...
    cmd_vel.linear.x = keyboard_cmd.linear.x
    cmd_vel.angular.z = keyboard_cmd.angular.z
    cmdvel_pub.publish(cmd_vel)
    #...
```

- Modificación del archivo `CMakeList.txt` del paquete:

```TypeScript
...
catkin_install_python(PROGRAMS scripts/<node_name_file>.py
  DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
)
...
```

- Compilar el paquete, desde la raíz del espacio de trabajo `catkin_ws`:

```bash
~/catkin_ws $ catkin_make
```

- Agregar en el archivo `.bashrc` la dirección de la carpeta de compilación.

```shell
~$ nano ~/.bashrc
-> source ~/catkin_ws/devel/setup.bash
$ source ~/.bashrc
```

### Verificación de aplicación:

1. Terminal No. 1 Inicializar el ROS-Master

```bash
$ roscore
```

2. Terminal No. 2. Iniciar el simulador `Turtlesim`.

```bash
$ rosrun turtlesim turtlesim_node
```

3. Terminal No. 3. Iniciar el nodo `turtle_node_gYY`

```bash
$ rosrun turtle_pkg_gYY turtle_node_gYY_teleop.py
```

## Teleoperación con joystick

1. Terminal No. 1 Inicializar el ROS-Master

```bash
$ roscore
```

2. Terminal No. 2. Iniciar nodo para lectura de joystick. 
 
 - Preconfiguración: modificación de la propiedades

```bash
$ ls -l /dev/input/
$ sudo chmod a+rw /dev/input/eventX
$ sudo chmod a+rw /dev/input/jsX
```
 - Definir la entrada (*input*) como parámetro del nodo.

```bash
$ rosparam set joy_node/dev "/dev/input/jsX"
```
 - Ejecutar nodo Joy

```bash
$ rosrun joy joy_node
```

3. Terminal No. 3. Leer *node_info* y *topic_info*


### Nodo turtle_joy (Python)

- Copiar el **nodo** `turtle_node_g00.py` asignarle el nombre `turtle_node_g00_joy.py`

- Inclusión de librerías en el programa.

```python
from sensor_msgs.msg import Joy

# def twist variable
joy_cmd = Twist()
```

- En la parte incial de la función del *node*, crear el subscriptor al nodo teleop.

```python
cmdjoy_subs = rospy.Subscriber('/joy',Joy,joy_callback)
```

- Creación de una función (*method*) encargada de leer el mensaje (`joy_callback`).

```python
def joy_callback(data):
    global joy_cmd
    '''Translate PS4 buttons into speed and spin
    Just use the left joystick (for now):
    Joystick (left)  left/right  axes[0]     +1 (left) to -1 (right)
    Joystick (rigth) up/down     axes[4]     +1 (up) to -1 (back)
    LB              buttons[5]  1 pressed, 0 otherwise
    '''
    if abs(data.axes[0]) > 0.10:
        joy_cmd.angular.z = data.axes[0]
    else:
        joy_cmd.angular.z = 0.0

    if abs(data.axes[4]) > 0.10:
        joy_cmd.linear.x = data.axes[4]
    else:
        joy_cmd.linear.x = 0.0
```

- Modificar la estructura del `While` para que contruya y envíe el mensaje.

```python
def turtle_node_g00(): #YY must change
global joy_cmd
#...
while not rospy.is_shutdown():
    #...
    cmd_vel.linear.x = joy_cmd.linear.x  #float(sys.argv[1])#
    cmd_vel.angular.z = joy_cmd.angular.z #float(sys.argv[2])#
    cmdvel_pub.publish(cmd_vel)
    #...
```

- Modificación del archivo `CMakeList.txt` del paquete:

```TypeScript
...
catkin_install_python(PROGRAMS scripts/<node_name_file>.py
  DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
)
...
```

- Compilar el paquete, desde la raíz del espacio de trabajo `catkin_ws`:

```bash
~/catkin_ws $ catkin_make
```

- Agregar en el archivo `.bashrc` la dirección de la carpeta de compilación.

```shell
~$ nano ~/.bashrc
-> source ~/catkin_ws/devel/setup.bash
$ source ~/.bashrc
```

### Verificación de aplicación:

1. Terminal No. 1 Inicializar el ROS-Master

```bash
$ roscore
```

2. Terminal No. 2. Iniciar el simulador `Turtlesim`.

```bash
$ rosrun turtlesim turtlesim_node
```

3. Terminal No. 3. Iniciar el nodo `turtle_node_gYY`

```bash
$ rosrun turtle_pkg_gYY turtle_node_gYY_joy.py
```